1. Reprenez le code de la dernière séance et créez une librairie.
2. Sérialisez le résultat du scraping en utilisant la librairie `pyserde`.

In [24]:
from importlib import reload

In [23]:
import lib04 as lib

In [25]:
reload(lib)

<module 'lib04' from 'C:\\Users\\perrollaz\\Documents\\lib04.py'>

In [76]:
import re

In [26]:
page = lib.recupere_page("https://fr.wikipedia.org/wiki/CAC_40")

In [28]:
table_1 = lib.recupere_premiere_table(page)

In [29]:
table_1

'<table><tbody><tr class=""><th scope="row">Place boursière</th><td class=""><div>\n<span class="flagicon"><a href="/wiki/Fichier:Flag_of_France.svg" class="image" title="Drapeau : France"><img alt="" src="//upload.wikimedia.org/wikipedia/commons/thumb/c/c3/Flag_of_France.svg/20px-Flag_of_France.svg.png" decoding="async" width="20" height="13" class="noviewer thumbborder" srcset="//upload.wikimedia.org/wikipedia/commons/thumb/c/c3/Flag_of_France.svg/30px-Flag_of_France.svg.png 1.5x, //upload.wikimedia.org/wikipedia/commons/thumb/c/c3/Flag_of_France.svg/40px-Flag_of_France.svg.png 2x" data-file-width="900" data-file-height="600" /></a></span> <a href="/wiki/Bourse_de_Paris" title="Bourse de Paris">Bourse de Paris</a>, <a href="/wiki/Paris" title="Paris">Paris</a></div></td></tr><tr class=""><th scope="row">Type</th><td class=""><div>\nIndice représentatif de l\'économie nationale</div></td></tr><tr class=""><th scope="row">Opérateur</th><td class=""><div>\n<a href="/wiki/Euronext" title

In [30]:
for i, table in enumerate(lib.recupere_tables(page)):
    print(i)

0
1
2
3
4
5
6
7


In [32]:
from bs4 import BeautifulSoup as BS

In [35]:
soupe = BS(page)

In [36]:
type(soupe)

bs4.BeautifulSoup

**EXERCICE** regarder les méthodes des objets `BeautifulSoup` commençant pas `find_` et utiliser celles qui vous semblent utiles.

In [38]:
tables = soupe.find_all("table")

In [40]:
type(tables)

bs4.element.ResultSet

In [41]:
len(tables)

8

In [52]:
table_cac40 = tables[2]

In [56]:
lignes = table_cac40.find_all("tr")

In [57]:
len(lignes)

41

In [58]:
header, *corps = lignes

In [59]:
header

<tr>
<th width="15%">Société
</th>
<th width="20%">Secteur
</th>
<th data-sort-type="number" width="9%">Poids indiciel (en %)<sup class="reference" id="cite_ref-59"><a href="#cite_note-59"><span class="cite_crochet">[</span>27<span class="cite_crochet">]</span></a></sup>
</th>
<th data-sort-type="number" width="9%"><a href="/wiki/Chiffre_d%27affaires" title="Chiffre d'affaires">Chiffre d'affaires</a> 2020 (en milliards d'euros)<sup class="reference" id="cite_ref-60"><a href="#cite_note-60"><span class="cite_crochet">[</span>28<span class="cite_crochet">]</span></a></sup>
</th>
<th data-sort-type="number" width="9%"><a href="/wiki/Capitalisation_boursi%C3%A8re" title="Capitalisation boursière">Capitalisation boursière</a> (en milliards d'euros)<sup class="reference" id="cite_ref-61"><a href="#cite_note-61"><span class="cite_crochet">[</span>29<span class="cite_crochet">]</span></a></sup>
</th>
<th width="9%">Entrée dans l'indice
</th></tr>

In [60]:
corps[0]

<tr>
<td><a href="/wiki/Air_liquide" title="Air liquide">Air liquide</a>
</td>
<td><a href="/wiki/Gaz_industriel" title="Gaz industriel">Gaz industriel</a>
</td>
<td>3,50
</td>
<td>20,49
</td>
<td>66,56
</td>
<td><span data-sort-value="19871231 !"></span><time class="nowrap date-lien" data-sort-value="1987-12-31" datetime="1987-12-31"><a href="/wiki/31_d%C3%A9cembre" title="31 décembre">31</a> <a href="/wiki/D%C3%A9cembre_1987" title="Décembre 1987">décembre</a> <a href="/wiki/1987" title="1987">1987</a></time>
</td></tr>

In [62]:
tds = corps[0].find_all("td")
tds

[<td><a href="/wiki/Air_liquide" title="Air liquide">Air liquide</a>
 </td>,
 <td><a href="/wiki/Gaz_industriel" title="Gaz industriel">Gaz industriel</a>
 </td>,
 <td>3,50
 </td>,
 <td>20,49
 </td>,
 <td>66,56
 </td>,
 <td><span data-sort-value="19871231 !"></span><time class="nowrap date-lien" data-sort-value="1987-12-31" datetime="1987-12-31"><a href="/wiki/31_d%C3%A9cembre" title="31 décembre">31</a> <a href="/wiki/D%C3%A9cembre_1987" title="Décembre 1987">décembre</a> <a href="/wiki/1987" title="1987">1987</a></time>
 </td>]

In [64]:
tds[0]

<td><a href="/wiki/Air_liquide" title="Air liquide">Air liquide</a>
</td>

In [65]:
tds[0].contents

[<a href="/wiki/Air_liquide" title="Air liquide">Air liquide</a>, '\n']

In [66]:
tds[0].text

'Air liquide\n'

In [68]:
a, *_ = tds[0].contents

In [70]:
a.attrs

{'href': '/wiki/Air_liquide', 'title': 'Air liquide'}

In [71]:
a.attrs["href"]

'/wiki/Air_liquide'

In [72]:
a.text

'Air liquide'

In [93]:
def genere_Entreprise(ligne):
    colonnes = ligne.find_all("td")
    nom = colonnes[0].contents[0].text
    lien = colonnes[0].contents[0].attrs["href"]
    secteur = colonnes[1].contents[0].text
    motif_nombre = re.compile("([0-9]+),([0-9]+)")
    pre_virgule, post_virgule = motif_nombre.match(colonnes[2].text).groups()
    poids_indiciel = (
        int(pre_virgule)  
        + int(post_virgule) / 10 ** len(post_virgule)
    )
    pre_virgule, post_virgule = motif_nombre.match(colonnes[3].text).groups()
    chiffre_affaire_euros = (
        int(pre_virgule) * 10 ** 9 
        + int(post_virgule) * 10 ** (9 - len(post_virgule))
    )
    pre_virgule, post_virgule = motif_nombre.match(colonnes[4].text).groups()
    capitalisation_euros = (
        int(pre_virgule) * 10 ** 9 
        + int(post_virgule) * 10 ** (9 - len(post_virgule))
    )
    motif_introduction = re.compile(".*datetime=\"([0-9]+)-([0-9]+)-([0-9]+)\".*")
    date_entree = [int(nombre) for nombre in motif_introduction.match(colonnes[5].prettify()).groups()]
    return lib.Entreprise(
        nom,
        lien,
        secteur,
        poids_indiciel,
        chiffre_affaire_euros,
        capitalisation_euros,
        date_entree,
    )

In [94]:
len(corps)

40

In [95]:
resultat = [genere_Entreprise(ligne) for ligne in corps]

AttributeError: 'NoneType' object has no attribute 'groups'

In [96]:
tds[5]

<td><span data-sort-value="19871231 !"></span><time class="nowrap date-lien" data-sort-value="1987-12-31" datetime="1987-12-31"><a href="/wiki/31_d%C3%A9cembre" title="31 décembre">31</a> <a href="/wiki/D%C3%A9cembre_1987" title="Décembre 1987">décembre</a> <a href="/wiki/1987" title="1987">1987</a></time>
</td>

In [87]:
tds[5].text

'31 décembre 1987\n'

In [88]:
tds[5].contents

[<span data-sort-value="19871231 !"></span>,
 <time class="nowrap date-lien" data-sort-value="1987-12-31" datetime="1987-12-31"><a href="/wiki/31_d%C3%A9cembre" title="31 décembre">31</a> <a href="/wiki/D%C3%A9cembre_1987" title="Décembre 1987">décembre</a> <a href="/wiki/1987" title="1987">1987</a></time>,
 '\n']

In [89]:
dir(tds[5])

['__bool__',
 '__call__',
 '__class__',
 '__contains__',
 '__copy__',
 '__delattr__',
 '__delitem__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setitem__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__unicode__',
 '__weakref__',
 '_all_strings',
 '_find_all',
 '_find_one',
 '_is_xml',
 '_lastRecursiveChild',
 '_last_descendant',
 '_should_pretty_print',
 'append',
 'attrs',
 'can_be_empty_element',
 'cdata_list_attributes',
 'childGenerator',
 'children',
 'clear',
 'contents',
 'decode',
 'decode_contents',
 'decompose',
 'descendants',
 'encode',
 'encode_contents',
 'extend',
 'extract',
 'fetchNextSiblings',
 'fetchParents',
 'fetchPrevious',
 'fetchPreviousSiblings',
 'find',
 'findAl

In [90]:
tds[5].string

In [92]:
type(tds[5].prettify())

str

# Apparté sur les générateurs/itérateurs

In [1]:
def essai():
    return 1
    return 2

In [5]:
type(essai)

function

In [2]:
essai()

1

In [3]:
essai()

1

In [11]:
def bis():
    yield 1
    yield 2
    yield 3

In [6]:
type(bis)

function

In [7]:
bis()

<generator object bis at 0x0000021D923BC8C8>

In [8]:
b = bis()

In [9]:
next(b)

1

In [10]:
next(b)

2

In [12]:
for i in bis():
    print(i)

1
2
3


In [13]:
def fibonacci():
    a, b = 0, 1
    while True:
        yield a
        a, b = b, a + b

In [14]:
f = fibonacci()

In [15]:
next(f)

0

In [16]:
next(f)

1

In [17]:
next(f)

1

In [18]:
next(f)

2

In [19]:
next(f)

3

In [20]:
next(f)

5

In [21]:
next(f)

8

In [22]:
for i in fibonacci():
    if i >= 1000:
        break
    print(i)

0
1
1
2
3
5
8
13
21
34
55
89
144
233
377
610
987
